## Global Settings and Import

In [1]:
import sys, os
sys.path.append("../../")
import numpy as np
import lightgbm as lgb
import papermill as pm
import pandas as pd
import category_encoders as ce
from tempfile import TemporaryDirectory
from sklearn.metrics import roc_auc_score, log_loss

import import_ipynb
import lightgbm_utils as lgb_utils
#import reco_utils.dataset.criteo as criteo

print("System version: {}".format(sys.version))
print("LightGBM version: {}".format(lgb.__version__))

importing Jupyter notebook from lightgbm_utils.ipynb
System version: 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
LightGBM version: 3.0.0


## Parameter Setting

In [2]:

MAX_LEAF = 64
MIN_DATA = 20
NUM_OF_TREES = 100
TREE_LEARNING_RATE = 0.15
EARLY_STOPPING_ROUNDS = 20
METRIC = "auc"
SIZE = "sample"

In [3]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'num_class': 1,
    'objective': "binary",
    'metric': METRIC,
    'num_leaves': MAX_LEAF,
    'min_data': MIN_DATA,
    'boost_from_average': True,
    #set it according to your cpu cores.
    'num_threads': 20,
    'feature_fraction': 0.8,
    'learning_rate': TREE_LEARNING_RATE,
}

## Import Data

In [4]:
import os
import logging
import requests
import math
import zipfile
from contextlib import contextmanager
from tempfile import TemporaryDirectory
from tqdm import tqdm

log = logging.getLogger(__name__)


def maybe_download(url, filename=None, work_directory=".", expected_bytes=None):
   
    if filename is None:
        filename = url.split("/")[-1]
    os.makedirs(work_directory, exist_ok=True)
    filepath = os.path.join(work_directory, filename)
    if not os.path.exists(filepath):

        r = requests.get(url, stream=True)
        total_size = int(r.headers.get("content-length", 0))
        block_size = 1024
        num_iterables = math.ceil(total_size / block_size)

        with open(filepath, "wb") as file:
            for data in tqdm(
                r.iter_content(block_size),
                total=num_iterables,
                unit="KB",
                unit_scale=True,
            ):
                file.write(data)
    else:
        log.info("File {} already downloaded".format(filepath))
    if expected_bytes is not None:
        statinfo = os.stat(filepath)
        if statinfo.st_size != expected_bytes:
            os.remove(filepath)
            raise IOError("Failed to verify {}".format(filepath))

    return filepath


@contextmanager
def download_path(path=None):
    """Return a path to download data. If `path=None`, then it yields a temporal path that is eventually deleted, 
    otherwise the real path of the input. 
    Args:
        path (str): Path to download data.
    Returns:
        str: Real path where the data is stored.
    Examples:
        >>> with download_path() as path:
        >>> ... maybe_download(url="http://example.com/file.zip", work_directory=path)
    """
    if path is None:
        tmp_dir = TemporaryDirectory()
        try:
            yield tmp_dir.name
        finally:
            tmp_dir.cleanup()
    else:
        path = os.path.realpath(path)
        yield path


def unzip_file(zip_src, dst_dir, clean_zip_file=True):
    """Unzip a file
    Args:
        zip_src (str): Zip file.
        dst_dir (str): Destination folder.
        clean_zip_file (bool): Whether or not to clean the zip file.
    """
    fz = zipfile.ZipFile(zip_src, "r")
    for file in fz.namelist():
        fz.extract(file, dst_dir)
    if clean_zip_file:
        os.remove(zip_src)

In [5]:
import pandas as pd
import os
import tarfile

try:
    from pyspark.sql.types import StructType, StructField, IntegerType, StringType
except ImportError:
    pass  # so the environment without spark doesn't break

#from reco_utils.dataset.download_utils import maybe_download, download_path
#from reco_utils.common.notebook_utils import is_databricks


CRITEO_URL = {
    "full": "https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz",
    "sample": "http://labs.criteo.com/wp-content/uploads/2015/04/dac_sample.tar.gz",
}
DEFAULT_HEADER = (
    ["label"]
    + ["int{0:02d}".format(i) for i in range(13)]
    + ["cat{0:02d}".format(i) for i in range(26)]
)


def load_pandas_df(size="sample", local_cache_path=None, header=DEFAULT_HEADER):
    
    with download_path(local_cache_path) as path:
        filepath = download_criteo(size, path)
        filepath = extract_criteo(size, filepath)
        df = pd.read_csv(filepath, sep="\t", header=None, names=header)
    return df


def load_spark_df(
    spark,
    size="sample",
    header=DEFAULT_HEADER,
    local_cache_path=None,
    dbfs_datapath="dbfs:/FileStore/dac",
    dbutils=None,
):
  
    with download_path(local_cache_path) as path:
        filepath = download_criteo(size, path)
        filepath = extract_criteo(size, filepath)

        if is_databricks():
            try:
                # Driver node's file path
                node_path = "file:" + filepath
                ## needs to be on dbfs to load
                dbutils.fs.cp(node_path, dbfs_datapath, recurse=True)
                path = dbfs_datapath
            except:
                raise ValueError(
                    "To use on a Databricks notebook, dbutils object should be passed as an argument"
                )
        else:
            path = filepath

        schema = get_spark_schema(header)
        df = spark.read.csv(path, schema=schema, sep="\t", header=False)
        df.cache().count()  # trigger execution to overcome spark's lazy evaluation
    return df


def download_criteo(size="sample", work_directory="."):
    """Download criteo dataset as a compressed file.
    Args:
        size (str): Size of criteo dataset. It can be "full" or "sample".
        work_directory (str): Working directory.
    Returns:
        str: Path of the downloaded file.
    """
    url = CRITEO_URL[size]
    return maybe_download(url, work_directory=work_directory)


def extract_criteo(size, compressed_file, path=None):
    """Extract Criteo dataset tar.
    Args:
        size (str): Size of Criteo dataset. It can be "full" or "sample".
        compressed_file (str): Path to compressed file.
        path (str): Path to extract the file.
    
    Returns:
        str: Path to the extracted file.
    
    """
    if path is None:
        folder = os.path.dirname(compressed_file)
        extracted_dir = os.path.join(folder, "dac")
    else:
        extracted_dir = path

    with tarfile.open(compressed_file) as tar:
        tar.extractall(extracted_dir)

    filename_selector = {"sample": "dac_sample.txt", "full": "train.txt"}
    return os.path.join(extracted_dir, filename_selector[size])


def get_spark_schema(header=DEFAULT_HEADER):
    ## create schema
    schema = StructType()
    ## do label + ints
    n_ints = 14
    for i in range(n_ints):
        schema.add(StructField(header[i], IntegerType()))
    ## do categoricals
    for i in range(26):
        schema.add(StructField(header[i + n_ints], StringType()))
    return schema


In [6]:
#from dataload import maybe_download, download_path, load_pandas_df, load_spark_df, download_criteo,extract_criteo

nume_cols = ["I" + str(i) for i in range(1, 14)]
cate_cols = ["C" + str(i) for i in range(1, 27)]
label_col = "Label"

header = [label_col] + nume_cols + cate_cols
with TemporaryDirectory() as tmp:
    all_data = load_pandas_df(size=SIZE, local_cache_path=tmp, header=header)
display(all_data.head())

100%|████████████████████████████████████████████████████████████████████████████| 8.58k/8.58k [00:03<00:00, 2.83kKB/s]


,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [7]:
all_data.to_csv (r'C:\Users\kaviy\Desktop\Assignment4\Part1_lightGBM\criteo_data.csv', index = False, header=True)

In [30]:
# split data to 3 sets    
length = len(all_data)
train_data = all_data.loc[:0.8*length-1]
valid_data = all_data.loc[0.8*length:0.9*length-1]
test_data = all_data.loc[0.9*length:]

## Basic Usage
### Ordinal Encoding

In [32]:
ord_encoder = ce.ordinal.OrdinalEncoder(cols=cate_cols)

def encode_csv(df, encoder, label_col, typ='fit'):
    if typ == 'fit':
        df = encoder.fit_transform(df)
    else:
        df = encoder.transform(df)
    y = df[label_col].values
    del df[label_col]
    return df, y

train_x, train_y = encode_csv(train_data, ord_encoder, label_col)
valid_x, valid_y = encode_csv(valid_data, ord_encoder, label_col, 'transform')
test_x, test_y = encode_csv(test_data, ord_encoder, label_col, 'transform')

print('Train Data Shape: X: {trn_x_shape}; Y: {trn_y_shape}.\nValid Data Shape: X: {vld_x_shape}; Y: {vld_y_shape}.\nTest Data Shape: X: {tst_x_shape}; Y: {tst_y_shape}.\n'
      .format(trn_x_shape=train_x.shape,
              trn_y_shape=train_y.shape,
              vld_x_shape=valid_x.shape,
              vld_y_shape=valid_y.shape,
              tst_x_shape=test_x.shape,
              tst_y_shape=test_y.shape,))
train_x.head()

Train Data Shape: X: (80000, 39); Y: (80000,).
Valid Data Shape: X: (10000, 39); Y: (10000,).
Test Data Shape: X: (10000, 39); Y: (10000,).



,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,1.0,...,1,1,1,1,1,1,1,1,1,1
1,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,1.0,...,2,2,1,2,2,1,1,2,1,2
2,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,1.0,...,3,3,2,3,3,2,1,3,2,3
3,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,NaN,...,4,4,2,3,4,1,1,4,2,3
4,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,1.0,...,4,5,2,3,5,1,2,5,2,3


## Create Model

In [33]:
lgb_train = lgb.Dataset(train_x, train_y.reshape(-1), params=params, categorical_feature=cate_cols)
lgb_valid = lgb.Dataset(valid_x, valid_y.reshape(-1), reference=lgb_train, categorical_feature=cate_cols)
lgb_test = lgb.Dataset(test_x, test_y.reshape(-1), reference=lgb_train, categorical_feature=cate_cols)
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=NUM_OF_TREES,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      valid_sets=lgb_valid,
                      categorical_feature=cate_cols)

[LightGBM] [Info] Number of positive: 17958, number of negative: 62042
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.048249 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38971
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.224475 -> initscore=-1.239776
[LightGBM] [Info] Start training from score -1.239776
[1]	valid_0's auc: 0.723997
Training until validation scores don't improve for 20 rounds
[2]	valid_0's auc: 0.736596
[3]	valid_0's auc: 0.740862
[4]	valid_0's auc: 0.745639
[5]	valid_0's auc: 0.74944
[6]	valid_0's auc: 0.751076
[7]	valid_0's auc: 0.752973
[8]	valid_0's auc: 0.753746
[9]	valid_0's auc: 0.7546
[10]	valid_0's auc: 0.755247
[11]	valid_0's auc: 0.7565
[12]	valid_0's auc: 0.756926
[13]	valid_0's auc: 0.757582
[14]	valid_0's auc: 0.757758
[15]	valid_0's auc: 0.758596
[16]	valid_0's auc: 

In [35]:
test_preds = lgb_model.predict(test_x)
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
res_basic = {"auc": auc, "logloss": logloss}
print(res_basic)
#pm.record("res_basic", res_basic)

{'auc': 0.7655408801711783, 'logloss': 0.4682583178835999}


## Optimized Usage
### Label-encoding and Binary-encoding

In [41]:
label_col = 'Label'
num_encoder = lgb_utils.NumEncoder(cate_cols, nume_cols, label_col)
train_x, train_y = num_encoder.fit_transform(train_data)
valid_x, valid_y = num_encoder.transform(valid_data)
test_x, test_y = num_encoder.transform(test_data)
del num_encoder
print('Train Data Shape: X: {trn_x_shape}; Y: {trn_y_shape}.\nValid Data Shape: X: {vld_x_shape}; Y: {vld_y_shape}.\nTest Data Shape: X: {tst_x_shape}; Y: {tst_y_shape}.\n'
      .format(trn_x_shape=train_x.shape,
              trn_y_shape=train_y.shape,
              vld_x_shape=valid_x.shape,
              vld_y_shape=valid_y.shape,
              tst_x_shape=test_x.shape,
              tst_y_shape=test_y.shape,))

2020-11-10 15:47:50,024 [INFO] Filtering and fillna features
100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 466.76it/s]
2020-11-10 15:47:51,727 [INFO] Ordinal encoding cate features
2020-11-10 15:47:52,645 [INFO] Target encoding cate features
100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:04<00:00,  6.26it/s]
2020-11-10 15:47:56,804 [INFO] Start manual binary encoding
100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:02<00:00,  9.64it/s]
2020-11-10 15:48:02,996 [INFO] Filtering and fillna features
100%|████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 3249.65it/s]
2020-11-10 15:48:03,121 [INFO] Ordinal encoding cate features
2020-11-10 15:48:03,189 [INFO] Target encoding cate features
100%|██████████████████████████████████████████████████████████████████████████████████| 26/

Train Data Shape: X: (80000, 268); Y: (80000, 1).
Valid Data Shape: X: (10000, 268); Y: (10000, 1).
Test Data Shape: X: (10000, 268); Y: (10000, 1).



## Training and Evaluation

In [42]:
lgb_train = lgb.Dataset(train_x, train_y.reshape(-1), params=params)
lgb_valid = lgb.Dataset(valid_x, valid_y.reshape(-1), reference=lgb_train)
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=NUM_OF_TREES,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      valid_sets=lgb_valid)

[LightGBM] [Info] Number of positive: 17958, number of negative: 62042
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.096549 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15787
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 267
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.224475 -> initscore=-1.239776
[LightGBM] [Info] Start training from score -1.239776
[1]	valid_0's auc: 0.727035
Training until validation scores don't improve for 20 rounds
[2]	valid_0's auc: 0.745243
[3]	valid_0's auc: 0.749993
[4]	valid_0's auc: 0.750781
[5]	valid_0's auc: 0.753203
[6]	valid_0's auc: 0.754548
[7]	valid_0's auc: 0.756143
[8]	valid_0's auc: 0.757718
[9]	valid_0's auc: 0.758371
[10]	valid_0's auc: 0.759261
[11]	valid_0's auc: 0.760498
[12]	valid_0's auc: 0.761402
[13]	valid_0's auc: 0.762118
[14]	valid_0's auc: 0.762916
[15]	valid_0's auc: 0.764051
[16]	valid_0's

In [44]:
test_preds = lgb_model.predict(test_x)
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
res_optim = {"auc": auc, "logloss": logloss}
print(res_optim)
#pm.record("res_optim", res_optim)

{'auc': 0.7758548016657666, 'logloss': 0.46030887404896165}


## Model saving and loading

In [45]:
with TemporaryDirectory() as tmp:
    save_file = os.path.join(tmp, r'finished.model')
    lgb_model.save_model(save_file)
    loaded_model = lgb.Booster(model_file=save_file)

# eval the performance again
test_preds = loaded_model.predict(test_x)

auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds), eps=1e-12)
print({"auc": auc, "logloss": logloss})

{'auc': 0.7758548016657666, 'logloss': 0.46030887404896165}
